In [32]:
#!pip install numpy-stl
#!pip install open3d
#!pip install anvil-uplink
import os
import anvil.server
import anvil.mpl_util
anvil.server.connect("2UZHPINDXBUVK3QPMLYKCF7M-XAU7U2Q7C7WSRJL3")
import open3d as o3d
import numpy as np
from stl import mesh
import math
from typing import NamedTuple, List
from google.colab import files
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import shutil
from anvil.google.drive import app_files
from google.colab import drive
import time
drive.mount('/content/gdrive')



@anvil.server.callable
def downloadSTL(sizeX, sizeY, boundMinX, boundMaxX, boundMinY, boundMaxY, strExpression):
  
  #create mathematical expression that will evalue input string expression
  def mathExpression(x,y):
    return eval(strExpression)
  
  #define grid spacing in x and y, and calculate total number of points to be sampled
  dX = (boundMaxX-boundMinX)/sizeX
  dY = (boundMaxY-boundMinY)/sizeY
  totalPoints = sizeX*sizeY

  #preallocate array for point cloud
  vertices = np.empty((totalPoints,3))

  #iterate through discretization of specified 2d set, define z coordinate at each point
  for i in range(sizeX):
    for j in range(sizeY):
      x = i*dX+boundMinX
      y = j*dY+boundMinY
      z = mathExpression(x,y)
      #k takes 2d axis and represents on 1d axis
      k = (j)*sizeX+i
      vertices[k][0]= x
      vertices[k][1] = y
      vertices[k][2] = z
  
  #I am using library open3d to go from point cloud to stl
  #define cloud, generate normal vectors
  cloud = o3d.geometry.PointCloud()
  cloud.points = o3d.utility.Vector3dVector(vertices)
  cloud.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

  #now we will use library built-in ball pivoting algorithm to generate mesh

  #we want our ball radius to be slightly larger than distance between points
  distance = cloud.compute_nearest_neighbor_distance()
  distanceAverage = np.mean(distance)
  r = 2.5*distanceAverage

  #create mesh to export
  exporter = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(cloud,o3d.utility.DoubleVector([r, r * 2]))

  #some quick smoothing/artifact removal
  exporter.remove_degenerate_triangles()
  exporter.remove_duplicated_triangles()
  exporter.remove_duplicated_vertices()
  exporter.remove_non_manifold_edges()



  #now we delete the current mesh stored in the folder on drive, this is strictly for front end interface  
  delete_filepath = '/content/gdrive/MyDrive/stlForMachina/mesh.stl'

  #write mesh to stl file, hold to allow file to delete from drive
  o3d.io.write_triangle_mesh("mesh.stl", exporter)
  time.sleep(7)

  #move mesh to my drive so that it can be sent to user in front end
  colab_link = "/content/mesh.stl"
  gdrive_link = "/content/gdrive/MyDrive/stlForMachina/"
  shutil.copy(colab_link, gdrive_link)
  #hold to allow file to upload to drive
  time.sleep(7)

@anvil.server.callable
def viewCloud(sizeX, sizeY, boundMinX, boundMaxX, boundMinY, boundMaxY, strExpression):
  
  def mathExpression(x,y):
    return eval(strExpression)
  
  dX = (boundMaxX-boundMinX)/sizeX
  dY = (boundMaxY-boundMinY)/sizeY
  totalPoints = sizeX*sizeY

  vertices = np.empty((totalPoints,3))

  for i in range(sizeX):
    for j in range(sizeY):
      x = i*dX+boundMinX
      y = j*dY+boundMinY
      z = mathExpression(x,y)
      #store points in k axis with lexicographic ordering
      k = (j)*sizeX+i
      vertices[k][0]= x
      vertices[k][1] = y
      vertices[k][2] = z

  cloud = o3d.geometry.PointCloud()
  cloud.points = o3d.utility.Vector3dVector(vertices)
  cloud.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))


  #up until this point, these functions were the same



  #I will be displaying two views of the point cloud
  colors = (0.5, 0.5, 0.5) + np.asarray(cloud.normals) * 0.5
  figsize = plt.rcParams.get('figure.figsize')
  fig = plt.figure(figsize=(figsize[0] * 2, figsize[1]))
  #two plots at different angles
  ax1 = fig.add_subplot(1, 2, 1, projection = '3d')
  ax2 = fig.add_subplot(1, 2, 2, projection = '3d')
  ax1.axis("off")
  ax1.view_init(90, -90) # front view
  ax1.scatter(vertices[:,0], vertices[:,1], vertices[:,2], s=1, c=colors)
  ax2.axis("off")
  ax2.view_init(90 + 90, -90) # top view
  ax2.scatter(vertices[:,0], vertices[:,1], vertices[:,2], s=1, c=colors)
  #return the image form of our plots
  return anvil.mpl_util.plot_image()
  


#anvil.server.wait_forever()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [30]:
#test block
downloadSTL(200, 200, -10, 10, -10, 10, 'np.sin(x)+np.sin(y)')